<a href="https://colab.research.google.com/github/Mat11-py/NLP_project/blob/main/Final_NLP_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROYECTO FINAL
El desafío a realizarse consiste en realizar un estudios exploratorio de 3 modelos en la tarea de clasificación de emociones (https://github.com/fmplaza/EmoEvent/tree/master/splits). Este dataset contiene 8409 tweets anotados con una de las siguientes categorías: anger, sadness, joy, disgust, fear, surprise, offensive, other. Además, los tweets están relacionados a eventos particulares encontrados en Twitter.

# 1. IMPORTAR O DESCARGAR LIBRERÍAS

In [ ]:
!pip install transformers==4.47.1

In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
from datasets import Dataset, ClassLabel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GRU
from tensorflow.keras.optimizers import Adam
from transformers import AdamW, get_scheduler, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, logging, TrainingArguments

# 2. CARGA DE DATOS

In [ ]:
path = "/content/drive/MyDrive/PhawAi/NLP/EmoEvent-master/emoevent_en.csv"

In [ ]:
df = pd.read_csv(path,sep="\t")

# 3. PREPROCESAMIENTO


In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe(include="all")

In [ ]:
df["emotion"].unique()

In [ ]:
ea = df.emotion.value_counts().reset_index() # ea -> emotion analysis
ea

In [ ]:
ea.columns

In [ ]:
plt.pie(ea['count'], labels=ea['emotion'], autopct="%1.1f%%")
plt.title("Porcentaje de emociones")
plt.show()

In [ ]:
oa = df.offensive.value_counts().reset_index() # oa -> offensive analysis
oa

In [ ]:
plt.pie(oa['count'], labels=oa['offensive'], autopct="%1.1f%%")
plt.title("Porcentaje de tweets ofensivos")
plt.show()

In [ ]:
df.duplicated().sum()

In [ ]:
def limpiar_texto(texto):
    """
    Limpieza de los tweets
    """
    texto = texto.lower()  # todo minúscula
    texto = re.sub(r'http\S+', '', texto)  # quitar URLs
    texto = re.sub(r'@\w+', '', texto)  # quitar menciones
    texto = re.sub(r'#\w+', '', texto)  # quitar hashtags
    texto = re.sub(r'\d+', '', texto)  # quitar números
    texto = texto.translate(str.maketrans('', '', string.punctuation))  # quitar puntuación
    texto = re.sub(r'\s+', ' ', texto).strip()  # quitar espacios extras
    return texto

In [ ]:
df['clean_tweet'] = df['tweet'].apply(limpiar_texto)

In [ ]:
df.head()

In [ ]:
le = LabelEncoder() #objeto codificador de etiquetas


df['emotion_encoded'] = le.fit_transform(df['emotion'])#transformación

# mapeo de clases
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Label mapping:", label_mapping)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.shape[0]

In [ ]:
X = df['clean_tweet']
y = df['emotion_encoded']

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y, #compensación de equitas
    random_state=42
)

# Tamaño de los conjuntos
print("Tamaño del conjunto de entrenamiento:", len(X_train))
print("Tamaño del conjunto de prueba:", len(X_test))


# 4. MODELAMIENTO

In [ ]:
def visualizacion_resultados(history,n):
  """
  visualización de la evaluación según accuracy
  """
  epochs = [i for i in range(n)]
  fig, ax = plt.subplots(1,2)
  train_acc = history.history["accuracy"]
  train_loss = history.history["loss"]
  val_acc = history.history["val_accuracy"]
  val_loss = history.history["val_loss"]
  fig.set_size_inches(16,9)

  ax[0].plot(epochs, train_acc, "go-",label = "Entrenamiento accuracy")
  ax[0].plot(epochs, val_acc, "ro-",label = "Validación accuracy")
  ax[0].set_title("Entrenamiento y validación accuracy")
  ax[0].legend()
  ax[0].set_xlabel("Epochs")
  ax[0].set_ylabel("Accuracy")

  ax[1].plot(epochs, train_loss, "go-",label = "Entrenamiento loss")
  ax[1].plot(epochs, val_loss, "ro-",label = "Validación loss")
  ax[1].set_title("Entrenamiento y validación loss")
  ax[1].legend()
  ax[1].set_xlabel("Epochs")
  ax[1].set_ylabel("Loss")

  plt.show()

## 4.1 LSTM
LSTM es una arquitectura de red neuronal recurrente (RNN) diseñada para manejar secuencias y capturar dependencias a largo plazo en texto. En el caso de clasificación de emociones en tweets, LSTM permite analizar la estructura secuencial de las palabras y detectar patrones emocionales a lo largo del mensaje, incluso si hay palabras clave separadas por otras menos relevantes.

In [ ]:
# Hiperparámetros
vocab_size = 10000  # número máximo de palabras a considerar
max_length = 100     # longitud máxima del tweet
oov_token = "<OOV>" # palabras fuera de vocabulario

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)

tokenizer.fit_on_texts(X_train)

# Convertir textos a secuencias de enteros
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Rellenar las secuencias para que todas tengan la misma longitud
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post', truncating='post')

print("Texto original:", X_train.iloc[0])
print("Secuencia:", X_train_seq[0])
print("Secuencia padded:", X_train_pad[0])


In [ ]:
num_classes = len(df['emotion_encoded'].unique()) #Clases

# Codificación one-hot
y_train_cat = to_categorical(y_train, num_classes=num_classes)
y_test_cat = to_categorical(y_test, num_classes=num_classes)


In [ ]:
# Modelo
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length),
    Bidirectional(LSTM(150, return_sequences=False)),
    #Dense(10000, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])


In [ ]:
history = model.fit(
    X_train_pad,
    y_train_cat,
    validation_split=0.1,
    epochs=15,
    batch_size=32,
    verbose=1
)


In [ ]:
visualizacion_resultados(history,15)

In [ ]:
# Predicciones LSTM
y_pred_probs = model.predict(X_test_pad)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test_cat, axis=1)


print(classification_report(y_true, y_pred, target_names=le.classes_))

In [ ]:
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de Confusión - LSTM')
plt.show()


# 4.2 GRU
GRU es una variante más eficiente y simplificada de LSTM. Para la clasificación de emociones, GRU puede capturar patrones secuenciales de manera similar, pero con menor costo computacional, lo que lo hace atractivo cuando se dispone de recursos limitados o se requiere un entrenamiento más rápido.

In [ ]:
model_gru = Sequential([
    Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length),
    GRU(150, return_sequences=False),
    #Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model_gru.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

In [ ]:
history_gru = model_gru.fit(
    X_train_pad,
    y_train_cat,
    validation_split=0.1,
    epochs=15,
    batch_size=32,
    verbose=1
)


In [ ]:
visualizacion_resultados(history_gru,15)

In [ ]:
# Predicciones gru
y_pred_probs = model_gru.predict(X_test_pad)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test_cat, axis=1)


print(classification_report(y_true, y_pred, target_names=le.classes_))

In [ ]:
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de Confusión - GRU')
plt.show()

# 4.3 BERT - FINE TUNING
BERT es un modelo de lenguaje preentrenado basado en Transformers que entiende el contexto completo de una palabra al considerar tanto la izquierda como la derecha (bidireccionalidad). Para la clasificación de emociones en tweets, BERT es altamente efectivo porque ya ha sido entrenado con grandes corpus de texto, lo que le permite comprender matices emocionales, sinónimos, ambigüedad semántica, y expresiones propias del lenguaje natural.

In [ ]:
df_Bert = df[['clean_tweet', 'emotion_encoded']]
df_Bert.head()

In [ ]:
# Mapeo de las clases
label_names = df_Bert['emotion_encoded'].unique().tolist()
label_names.sort()  # Asegura el orden de los índices

In [ ]:
dataset = Dataset.from_pandas(df_Bert)

In [ ]:
# Convertir la columna de etiquetas a ClassLabel
dataset = dataset.cast_column("emotion_encoded", ClassLabel(names=[str(i) for i in label_names]))

dataset = dataset.train_test_split(test_size=0.2, stratify_by_column="emotion_encoded", seed=42)


In [ ]:
dataset

In [ ]:
#tokenizador
model_checkpoint = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def tokenize_function(example):
  """
  tokenizador en el dataset
  """
  return tokenizer(
        example["clean_tweet"],
        padding="max_length",
        truncation=True,
        max_length=128
        )

In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [ ]:
tokenized_dataset['train'][0]

In [ ]:
num_labels = tokenized_dataset['train'].features['emotion_encoded'].num_classes
num_labels

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=num_labels
)

In [ ]:
model.config

In [ ]:
tokenized_dataset

In [ ]:
# Eliminar columnas innecesarias
tokenized_dataset = tokenized_dataset.remove_columns(['clean_tweet','token_type_ids'])  # Ajusta si es necesario

# Establecer el formato para PyTorch
tokenized_dataset.set_format("torch")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="./bert-emotion",         # carpeta de salida
    evaluation_strategy="epoch",         # evalúa al final de cada época
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_dir='./logs',
    logging_steps=10,
    report_to="none",
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Entrenar
trainer.train()

In [ ]:
# Predicciones
predictions = trainer.predict(tokenized_dataset["test"])
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = predictions.label_ids



In [ ]:
# Reporte de clasificación
print(classification_report(y_true, y_pred, target_names=tokenized_dataset['train'].features['labels'].names))



In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=tokenized_dataset['train'].features['labels'].names,
            yticklabels=tokenized_dataset['train'].features['labels'].names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix - BERT")
plt.show()